# Custom MCP Agent with Data Automation

This notebook demonstrates how to create an AI agent that can intelligently ingest and analyze information from collaboration platforms like Microsoft Teams, Outlook, SharePoint, meeting recordings, and other ai data engine sources using a custom Model Context Protocol (MCP) server.

## Import the Needed Packages

We'll import all the necessary libraries for creating our custom Data Automation MCP-enabled agent.

In [ ]:
# Install required packages from requirements.txt
%pip install -r requirements.txt

In [1]:
import os
import asyncio
import json
from dotenv import load_dotenv

# Semantic Kernel imports
from azure.identity import DefaultAzureCredential
from semantic_kernel import Kernel
from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.contents import FunctionCallContent, FunctionResultContent, StreamingTextContent
from semantic_kernel.connectors.mcp import MCPStdioPlugin

from IPython.display import display, HTML

## Setting up Azure OpenAI Connection

Configure the Azure OpenAI service using Azure AD authentication.

In [2]:
load_dotenv()

# Option 1: Using API Key (recommended for development)
chat_completion_service = AzureChatCompletion(
    deployment_name=os.environ.get("AZURE_OPENAI_DEPLOYMENT_NAME", "gpt-4o-mini"),
    endpoint=os.environ.get("AZURE_OPENAI_ENDPOINT"),
    api_version=os.environ.get("AZURE_OPENAI_API_VERSION", "2024-02-01"),
    api_key=os.environ.get("AZURE_OPENAI_API_KEY")
)

# Option 2: Using Azure AD Authentication (uncomment to use)
# Create Azure credential 
credential = DefaultAzureCredential()

# Create a token provider function
def get_azure_ad_token():
    """Function to get Azure AD token for OpenAI."""
    token = credential.get_token("https://cognitiveservices.azure.com/.default")
    return token.token

# chat_completion_service = AzureChatCompletion(
#     deployment_name=os.environ.get("AZURE_OPENAI_DEPLOYMENT_NAME", "gpt-4o-mini"),
#     endpoint=os.environ.get("AZURE_OPENAI_ENDPOINT"),
#     api_version=os.environ.get("AZURE_OPENAI_API_VERSION", "2024-02-01"),
#     ad_token=get_azure_ad_token()

print("Azure OpenAI service configured successfully!")

Azure OpenAI service configured successfully!


## Configure Custom MCP Plugin

Set up the Model Context Protocol plugin to connect to our custom Data Automation server.

In [3]:
# Create MCP Plugin for stdio connection to our custom server
# Note: name must follow pattern ^[0-9A-Za-z_]+$
mcp_plugin = MCPStdioPlugin(
    name="data_automation_server",
    command="python",
    args=["server.py"],
    description="Data Automation tools for ai data engine platforms"
)

# Create kernel
kernel = Kernel()

print(f"MCP Plugin configured for custom Data Automation server")

MCP Plugin configured for custom Data Automation server


## Creating the Custom MCP Agent

Create an AI agent with access to our custom Data Automation tools through MCP.

In [4]:
# Create agent with MCP plugin via kernel
agent = ChatCompletionAgent(
    service=chat_completion_service,
    kernel=kernel,
    name="Data-Automation-Agent",
    instructions="""
    You are a Data Automation assistant that helps analyze and extract insights from 
    collaboration data across various platforms including Microsoft Teams, Outlook, SharePoint, 
    Slack, Discord, Zoom, and other ai data engine platforms.
    
    Your capabilities include:
    - Ingesting chat messages from collaboration platforms
    - Analyzing email data from various email platforms
    - Processing documents from SharePoint, OneDrive, Google Drive
    - Analyzing meeting recordings and transcripts
    - Searching across all ai data engine sources using natural language
    - Extracting action items and tasks from conversations
    - Providing collaboration insights and analytics
    
    Available MCP tools:
    - ingest_chat_messages: Ingest and analyze chat messages from ai data engine platforms
    - ingest_email_data: Ingest and analyze email data from email platforms
    - ingest_documents: Ingest and analyze documents from document platforms
    - ingest_meeting_recordings: Ingest and analyze meeting recordings and transcripts
    - search_ai_data: Search across all ingested ai data engine sources
    - get_collaboration_insights: Get insights about collaboration patterns
    - extract_action_items: Extract action items and tasks from ai data engine sources
    
    When providing analysis:
    1. Always show the data you retrieved and explain your reasoning
    2. Provide actionable insights and recommendations
    3. Highlight key patterns, trends, and important information
    4. Suggest follow-up actions when appropriate
    5. Maintain confidentiality and professional context
    
    Be helpful, analytical, and focus on providing valuable business insights from the ai data engine.
    """
)

print(f"Created agent: {agent.name}")

Created agent: Data-Automation-Agent


## Initialize Custom MCP Connection

Connect to our custom MCP server and load available tools.

In [5]:
async def initialize_custom_mcp():
    """Initialize custom MCP plugin connection and load tools."""
    try:
        # Connect to MCP server
        await mcp_plugin.connect()
        print(f"✓ Custom MCP Plugin connected successfully")
        
        # Load tools and prompts
        await mcp_plugin.load_tools()
        print("✓ Custom MCP tools loaded")
        
        # Add the MCP plugin to the kernel
        kernel.add_plugin(mcp_plugin)
        print(f"✓ Custom MCP plugin added to kernel")
        
        # Show available functions
        print("\nAvailable custom MCP tools:")
        for plugin_name, plugin in kernel.plugins.items():
            for function_name, function in plugin.functions.items():
                print(f"  - {function_name}: {function.description}")
        
        return True
        
    except Exception as e:
        print(f"Custom MCP Plugin setup error: {e}")
        import traceback
        traceback.print_exc()
        return False

# Initialize the custom MCP connection
mcp_initialized = await initialize_custom_mcp()
print(f"\nCustom MCP initialization {'successful' if mcp_initialized else 'failed'}")

✓ Custom MCP Plugin connected successfully
✓ Custom MCP tools loaded
✓ Custom MCP plugin added to kernel

Available custom MCP tools:
  - extract_action_items: Extract action items and tasks from ingested work tech data using AI analysis.
  - get_collaboration_insights: Get insights and analytics about collaboration patterns from ingested work tech data.
  - ingest_chat_messages: Ingest and analyze chat messages from work collaboration platforms like Teams, Slack, Discord.
  - ingest_documents: Ingest and analyze documents from SharePoint, OneDrive, Google Drive, and other document platforms.
  - ingest_email_data: Ingest and analyze email data from Outlook, Gmail, and other email platforms.
  - ingest_meeting_recordings: Ingest and analyze meeting recordings and transcripts from Teams, Zoom, and other platforms.
  - search_work_data: Search across all ingested work tech data using keywords or content search.

Custom MCP initialization successful


## Agent Interaction Function

Create a function to handle user interactions with the agent, displaying both the conversation and function calls.

In [6]:
async def chat_with_agent(user_input, thread=None):
    """Chat with the agent and display results with function call details."""
    
    html_output = (
        f"<div style='margin-bottom:10px'>"
        f"<div style='font-weight:bold'>User:</div>"
        f"<div style='margin-left:20px'>{user_input}</div></div>"
    )

    agent_name = None
    full_response: list[str] = []
    function_calls: list[str] = []

    # Buffer to reconstruct streaming function call
    current_function_name = None
    argument_buffer = ""

    try:
        async for response in agent.invoke_stream(
            messages=user_input,
            thread=thread,
        ):
            thread = response.thread
            agent_name = response.name
            content_items = list(response.items)

            for item in content_items:
                if isinstance(item, FunctionCallContent):
                    if item.function_name:
                        current_function_name = item.function_name

                    # Accumulate arguments (streamed in chunks)
                    if isinstance(item.arguments, str):
                        argument_buffer += item.arguments
                elif isinstance(item, FunctionResultContent):
                    # Finalize any pending function call before showing result
                    if current_function_name:
                        formatted_args = argument_buffer.strip()
                        try:
                            parsed_args = json.loads(formatted_args) if formatted_args else {}
                            formatted_args = json.dumps(parsed_args)
                        except Exception:
                            pass  # leave as raw string

                        function_calls.append(f"🔧 Calling function: {current_function_name}({formatted_args})")
                        current_function_name = None
                        argument_buffer = ""

                    function_calls.append(f"📊 Function Result:\n{item.result}")
                elif isinstance(item, StreamingTextContent) and item.text:
                    full_response.append(item.text)

        # Display function calls if any were made
        if function_calls:
            html_output += (
                "<div style='margin-bottom:10px'>"
                "<details>"
                "<summary style='cursor:pointer; font-weight:bold; color:#0066cc;'>MCP Tool Calls (click to expand)</summary>"
                "<div style='margin:10px; padding:10px; background-color:#f8f8f8; "
                "border:1px solid #ddd; border-radius:4px; white-space:pre-wrap; font-size:14px; color:#333;'>"
                f"{chr(10).join(function_calls)}"
                "</div></details></div>"
            )

        html_output += (
            "<div style='margin-bottom:20px'>"
            f"<div style='font-weight:bold'>{agent_name or 'Assistant'}:</div>"
            f"<div style='margin-left:20px; white-space:pre-wrap'>{''.join(full_response)}</div></div><hr>"
        )

        display(HTML(html_output))
        return thread
        
    except Exception as e:
        print(f"Error during agent execution: {e}")
        import traceback
        traceback.print_exc()
        return thread

print("Agent interaction function ready!")

Agent interaction function ready!


## Test the Data Automation Agent

Now let's test our custom MCP-enabled agent with some data automation questions.

In [7]:
# Test questions for the Data Automation agent
test_questions = [
    "Ingest all chat messages from Microsoft Teams and provide a sentiment analysis",
    "What are the current collaboration insights across all platforms?",
    "Extract action items from all ai data engine sources"
]

# Run the conversation
thread = None
for question in test_questions:
    thread = await chat_with_agent(question, thread)
    print("\n" + "="*80 + "\n")  # Separator between questions

In [8]:
# Detailed analysis question
detailed_question = "Search for any mentions of 'API' or 'database' across all ai data engine platforms and provide a comprehensive analysis"
thread = await chat_with_agent(detailed_question, thread)

## Interactive Chat

Use this cell to ask your own questions to the Data Automation agent. The agent will use the custom MCP tools to provide insights from ai data engine data.

In [ ]:
# Interactive chat - modify the question below
user_question = "What are the most discussed topics in our organization and which platforms are being used most?"

# Use the existing thread to maintain conversation context
thread = await chat_with_agent(user_question, thread)

## Custom Data Analysis

Create your own custom query to test the agent's data analysis capabilities.

In [ ]:
# Custom query - replace with your own question
custom_question = "Analyze our meeting patterns and extract any follow-up actions from recent meetings"

thread = await chat_with_agent(custom_question, thread)

## View Available MCP Tools

Let's examine what tools are available from our custom MCP server.

In [ ]:
# Display available tools and their descriptions
print("Available Custom MCP Tools:")
print("=" * 50)

for plugin_name, plugin in kernel.plugins.items():
    print(f"\nPlugin: {plugin_name}")
    for function_name, function in plugin.functions.items():
        print(f"  🔧 {function_name}")
        print(f"     Description: {function.description}")
        print(f"     Parameters: {[param.name for param in function.parameters]}")

## Cleanup

Clean up the MCP connection when done.

In [ ]:
# Clean up MCP connection
try:
    await mcp_plugin.close()
    print("✓ Custom MCP Plugin closed successfully")
except Exception as e:
    print(f"Error closing custom MCP plugin: {e}")